In [1]:
import os 
import pandas as pd 
import sys 
import re 

current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path,'..','..','..'))  # '..','..','..'
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    
from pipeline.plotting.TS_analysis import plot_TS
from jupyter_ipynb.NetMob_training_analysis.plotting import plot_boxplot_on_metric
from experiences.convert_df_to_latex import dataframe_to_latex,parse_index_exp1_2,parse_index_exp4
from experiences.convert_df_to_latex import update_df_metrics,load_csv
from experiences.convert_df_to_latex import tackle_trial_j as tackle_trial_for_distrib
from experiences.common_results import dic_exp_to_names,find_baseline,dic_trials

folder_path = '../../save/K_fold_validation/training_wo_HP_tuning'  # '../../save'


In [2]:
for exp_i,trials in dic_trials.items():
    if exp_i == 'set_experience_HERE' :#'Exp1_subway_in': #'set_experience_HERE': # set_experience_HERE # 'Exp1_subway_in'# 'Exp4_15min' # 'Exp4' # 'Exp3_bike_15min_h4' # 'Exp3' # 'Exp1'
        for trial_j in trials:
            df_j_all = pd.DataFrame()
            for n_bis in range(1,6):
                file_path = f"{folder_path}/{exp_i}/{dic_exp_to_names[exp_i]}/{trial_j}_bis{n_bis}/Losses_{trial_j}_bis{n_bis}.csv"
                df_j = pd.read_csv(file_path,index_col = 0)
                df_j.columns = [f'Train_{n_bis}',f'Valid_{n_bis}']
                df_j_all = pd.concat([df_j_all,df_j],axis=1)
            plot_TS(df_j_all,width=1500,height=400,bool_show=True,title=f"{exp_i}, {trial_j}",x_datetime = False)


In [7]:
import os 
import pandas as pd 
from pipeline.plotting.TS_analysis import plot_TS

def tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics,plot_losses):
    df_j_all = pd.DataFrame()
    metric_i = []
    for n_bis in range(1,6):
        df_j_all, metric_i = load_csv(folder_path,dic_exp_to_names,exp_i,trial_j,n_bis,df_j_all,metric_i,metrics)

    metric_i = pd.DataFrame(pd.DataFrame(metric_i).agg(['mean','std']).unstack()).T
    metric_i.index = [f"{trial_j}"]
    L_metrics.append(metric_i)
    if plot_losses:
            plot_TS(df_j_all,width=1500,height=400,bool_show=True,title=f"{exp_i}, {trial_j}",x_datetime = False)
    return L_metrics

folder_path = '../../../save/K_fold_validation/training_wo_HP_tuning' # '../../../'  or ''
metrics = ['rmse','mae','mase']
plot_losses = False 
experiences = ['Exp1_subway_out']# ['Exp1_subway_in','Exp1_subway_out','Exp1']
for exp_i in experiences:
    trials = dic_trials[exp_i]
    print(f"\n------------------ {exp_i} ---")
    
    L_metrics = []
    for trial_j in trials:
        L_metrics = tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics,plot_losses)

    df_metrics_all = pd.concat(L_metrics)

    horizons = list(set([c[0].split('_')[-1][1:] for c in df_metrics_all.columns]))
    for horizon in horizons:
        df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c[0].endswith(f"_h{horizon}")]].dropna()

        globals()[f"df_metrics_{exp_i}_h{horizon}"] = df_horizon.sort_values(by=[(f'rmse_h{horizon}','mean')])

        baseline_index = find_baseline(exp_i,h=horizon)
        for c in globals()[f"df_metrics_{exp_i}_h{horizon}"].columns:
            if 'mean' == c[1]:
                globals()[f"df_metrics_{exp_i}_h{horizon}"][c] = globals()[f"df_metrics_{exp_i}_h{horizon}"][c].apply(lambda x: f"{round(x,2)} ({round(100*(-1+x/globals()[f'df_metrics_{exp_i}_h{horizon}'].loc[baseline_index,c]),2)}\%)")
        
        display(globals()[f"df_metrics_{exp_i}_h{horizon}"])



------------------ Exp1_subway_out ---


rmse_h1            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_late_f...  35.38 (-3.88\%)  0.173060   
STAEformer_subway_out_subway_in_calendar_early_...  35.71 (-2.99\%)  0.204204   
STAEformer_subway_out_subway_in_calendar_early_...  35.81 (-2.73\%)  0.340023   
STAEformer_subway_out_subway_in_calendar_early_...  35.84 (-2.63\%)  0.157459   
STAEformer_subway_out_subway_in_calendar_late_f...  35.87 (-2.55\%)  0.169968   
STAEformer_subway_out_subway_in_calendar_early_...  36.05 (-2.06\%)  0.278188   
STAEformer_subway_out_calendar__e80_h1                36.81 (0.0\%)  0.160002   

                                                             mae_h1            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_late_f...  19.68 (-3.51\%)  0.111560   
STAEformer_subway_out_subway_in_calendar_early_...  19.64 (-3.68\%)  0.088888   
STAEformer_subway_out_subway_in_calendar_early_...  19.76 (-3.11\%)  0.110338   
STAEformer_subway_out_subway_in_calendar_early_...   19.8 (-2.93\%)  0.060690   
STAEformer_subway_out_subway_in_calendar_late_f...   20.1 (-1.42\%)  0.067312   
STAEformer_subway_out_subway_in_calendar_early_...  19.74 (-3.22\%)  0.069391   
STAEformer_subway_out_calendar__e80_h1                20.39 (0.0\%)  0.082622   

                                                           mase_h1            
                                                              mean       std  
STAEformer_subway_out_subway_in_calendar_late_f...  0.61 (-3.06\%)  0.003469  
STAEformer_subway_out_subway_in_calendar_early_...  0.61 (-3.23\%)  0.002769  
STAEformer_subway_out_subway_in_calendar_early_...  0.61 (-2.65\%)  0.003437  
STAEformer_subway_out_subway_in_calendar_early_...  0.62 (-2.47\%)  0.001888  
STAEformer_subway_out_subway_in_calendar_late_f...  0.63 (-0.96\%)  0.002089  
STAEformer_subway_out_subway_in_calendar_early_...  0.61 (-2.77\%)  0.002160  
STAEformer_subway_out_calendar__e80_h1                0.63 (0.0\%)  0.002557

rmse_h4            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_early_...  40.59 (-4.14\%)  0.565579   
STAEformer_subway_out_subway_in_calendar_early_...  40.81 (-3.63\%)  0.313657   
STAEformer_subway_out_subway_in_calendar_early_...  40.88 (-3.45\%)  0.292779   
STAEformer_subway_out_subway_in_calendar_early_...  41.11 (-2.91\%)  0.184949   
STAEformer_subway_out_subway_in_calendar_late_f...  41.88 (-1.09\%)  0.500659   
STAEformer_subway_out_subway_in_calendar_late_f...  41.89 (-1.06\%)  0.148227   
STAEformer_subway_out_calendar__e80_h4                42.34 (0.0\%)  0.367245   

                                                             mae_h4            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_early_...   22.18 (-3.7\%)  0.180961   
STAEformer_subway_out_subway_in_calendar_early_...  22.55 (-2.08\%)  0.218737   
STAEformer_subway_out_subway_in_calendar_early_...  22.29 (-3.21\%)  0.155751   
STAEformer_subway_out_subway_in_calendar_early_...  22.55 (-2.06\%)  0.119015   
STAEformer_subway_out_subway_in_calendar_late_f...  22.85 (-0.75\%)  0.205196   
STAEformer_subway_out_subway_in_calendar_late_f...  22.95 (-0.32\%)  0.108604   
STAEformer_subway_out_calendar__e80_h4                23.03 (0.0\%)  0.209461   

                                                           mase_h4            
                                                              mean       std  
STAEformer_subway_out_subway_in_calendar_early_...   0.69 (-3.2\%)  0.005634  
STAEformer_subway_out_subway_in_calendar_early_...   0.7 (-1.58\%)  0.006807  
STAEformer_subway_out_subway_in_calendar_early_...  0.69 (-2.71\%)  0.004846  
STAEformer_subway_out_subway_in_calendar_early_...   0.7 (-1.55\%)  0.003705  
STAEformer_subway_out_subway_in_calendar_late_f...  0.71 (-0.24\%)  0.006391  
STAEformer_subway_out_subway_in_calendar_late_f...   0.71 (0.19\%)  0.003380  
STAEformer_subway_out_calendar__e80_h4                0.71 (0.0\%)  0.006488

In [25]:
exp_i = 'Exp1_subway_out' # 'Exp1_subway_out' #'Exp1_subway_in' # 'Exp1' 
metrics = ['rmse','mae','mase']
trials = dic_trials[exp_i]
print(f"\n------------------ {exp_i} ---")

L_metrics = []
for trial_j in trials:
    L_metrics = tackle_trial_for_distrib(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics)
df_metrics_all = pd.concat(L_metrics)
display(df_metrics_all)



L_metrics = []
for trial_j in trials:
    L_metrics = tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics,plot_losses)

df_metrics_all = pd.concat(L_metrics)
display(df_metrics_all)

# horizons = list(set([c.split('_')[-1][1:] for c in df_metrics_all.columns]))
# for horizon in horizons:
#     print('\n----------------')
#     print(f"Horizon: {horizon}")
#     df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c.endswith(f"_h{horizon}")]].dropna()
#     globals()[f"df_metrics_{exp_i}_h{horizon}"] = df_horizon.sort_values(by=[(f'rmse_h{horizon}','mean')])

#     baseline_index = find_baseline(exp_i,h=horizon)
#     for c in globals()[f"df_metrics_{exp_i}_h{horizon}"].columns:
#         if 'mean' == c[1]:
#             globals()[f"df_metrics_{exp_i}_h{horizon}"][c] = globals()[f"df_metrics_{exp_i}_h{horizon}"][c].apply(lambda x: f"{round(x,2)} ({round(100*(-1+x/globals()[f'df_metrics_{exp_i}_h{horizon}'].loc[baseline_index,c]),2)}\%)")
    
#     display(globals()[f"df_metrics_{exp_i}_h{horizon}"])


------------------ Exp1_subway_out ---


,rmse_h1,mae_h1,mase_h1,rmse_h4,mae_h4,mase_h4
STAEformer_subway_out_subway_in_calendar_early_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis1,35.936996,19.766682,0.615077,NaN,NaN,NaN
STAEformer_subway_out_subway_in_calendar_early_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis2,35.468311,19.534847,0.607851,NaN,NaN,NaN
STAEformer_subway_out_subway_in_calendar_early_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis3,35.887951,19.666235,0.611938,NaN,NaN,NaN
STAEformer_subway_out_subway_in_calendar_early_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis4,35.551296,19.581841,0.609323,NaN,NaN,NaN
STAEformer_subway_out_subway_in_calendar_early_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis5,35.707958,19.661261,0.611786,NaN,NaN,NaN
...,...,...,...,...,...,...
STAEformer_subway_out_subway_in_calendar_early_fusion_s_proj_t_proj__e80_h4_bis1,NaN,NaN,NaN,41.248383,22.824749,0.710561
STAEformer_subway_out_subway_in_calendar_early_fusion_s_proj_t_proj__e80_h4_bis2,NaN,NaN,NaN,40.416492,22.355501,0.695957
STAEformer_subway_out_subway_in_calendar_early_fusion_s_proj_t_proj__e80_h4_bis3,NaN,NaN,NaN,40.739902,22.362532,0.696160
STAEformer_subway_out_subway_in_calendar_early_fusion_s_proj_t_proj__e80_h4_bis4,NaN,NaN,NaN,40.960236,22.735710,0.707765


rmse_h1            \
                                                         mean       std   
STAEformer_subway_out_subway_in_calendar_early_...  35.710503  0.204204   
STAEformer_subway_out_subway_in_calendar_early_...  35.806162  0.340023   
STAEformer_subway_out_subway_in_calendar_late_f...  35.381439  0.173060   
STAEformer_subway_out_subway_in_calendar_late_f...  35.871664  0.169968   
STAEformer_subway_out_subway_in_calendar_late_f...        NaN       NaN   
STAEformer_subway_out_calendar__e80_h4                    NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...  36.050803  0.278188   
STAEformer_subway_out_subway_in_calendar_late_f...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...  35.840729  0.157459   
STAEformer_subway_out_calendar__e80_h1              36.809328  0.160002   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   

                                                       mae_h1            \
                                                         mean       std   
STAEformer_subway_out_subway_in_calendar_early_...  19.642173  0.088888   
STAEformer_subway_out_subway_in_calendar_early_...  19.759624  0.110338   
STAEformer_subway_out_subway_in_calendar_late_f...  19.676573  0.111560   
STAEformer_subway_out_subway_in_calendar_late_f...  20.103162  0.067312   
STAEformer_subway_out_subway_in_calendar_late_f...        NaN       NaN   
STAEformer_subway_out_calendar__e80_h4                    NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...  19.735289  0.069391   
STAEformer_subway_out_subway_in_calendar_late_f...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...  19.796138  0.060690   
STAEformer_subway_out_calendar__e80_h1              20.392890  0.082622   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   

                                                     mase_h1            \
                                                        mean       std   
STAEformer_subway_out_subway_in_calendar_early_...  0.611195  0.002769   
STAEformer_subway_out_subway_in_calendar_early_...  0.614853  0.003437   
STAEformer_subway_out_subway_in_calendar_late_f...  0.612266  0.003469   
STAEformer_subway_out_subway_in_calendar_late_f...  0.625547  0.002089   
STAEformer_subway_out_subway_in_calendar_late_f...       NaN       NaN   
STAEformer_subway_out_calendar__e80_h4                   NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...  0.614090  0.002160   
STAEformer_subway_out_subway_in_calendar_late_f...       NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...       NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...  0.615989  0.001888   
STAEformer_subway_out_calendar__e80_h1              0.631610  0.002557   
STAEformer_subway_out_subway_in_calendar_early_...       NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...       NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...       NaN       NaN   

                                                      rmse_h4            \
                                                         mean       std   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_early_...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_late_f...        NaN       NaN   
STAEformer_subway_out_subway_in_calendar_late_f...        NaN       NaN   
STAEformer_subway

In [18]:
# experiences = ['Exp1_subway_in','Exp1_subway_out','Exp1']
exp_i = 'Exp1_subway_out' # 'Exp1_subway_out' #'Exp1_subway_in' # 'Exp1' 
metrics = ['rmse','mae','mase']
trials = dic_trials[exp_i]
print(f"\n------------------ {exp_i} ---")

L_metrics = []
for trial_j in trials:
    L_metrics = tackle_trial_for_distrib(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics)
df_metrics_all = pd.concat(L_metrics)

def change_legend(row):
    if row.legend_group == 'Other Methods':
        if 'early_fusion' in row.name:
            return 'Early Fusion Other Methods '
        elif 'late_fusion' in row.name:
            return 'Late Fusion Other Methods '
        else:
            return row.legend_group
    else:
        return row.legend_group

def reduce_id(x):
    x = x.replace('late_fusion_','L_')
    x = x.replace('early_fusion_','E_')
    x = x.replace('backbone','BB')
    return x 


configs_to_keep = [
    'Baseline',
    # Model backbone : ---- 
    'late_fusion_traffic_model_backbone_In24_adp16',
    'early_fusion_traffic_model_backbone_In24_adp16',
    # CABB with adpQ = 0 ---
    'early_fusion_CABB_In24_ctxIn8_adp16_adpQ0',
    'late_fusion_CABB_In24_ctxIn8_adp16_adpQ0',
    # CABB with adpQ = 24 ---
    'early_fusion_CABB_In24_ctxIn8_adp16_adpQ24',
    'late_fusion_CABB_In24_ctxIn8_adp16_adpQ24',
    # 'early_fusion_CABB_In24_ctxIn24_adp16_adpQ24',
    # 'late_fusion_CABB_In24_ctxIn24_adp16_adpQ24',
    
    # Others ---
    'late_fusion_simple_embedding',
    'early_fusion_shared_embedding',
    'early_fusion_independant_embedding'   
] 
# configs_to_keep = None 

palette = ['#4e79a7','#f28e2b','#e15759', '#59a14f', '#c7e9c0','#1a4314' ]
legend_groups = ['Baseline','Independant Embedding','Shared Embedding','Early Fusion Other Methods ','Late Fusion Other Methods ']
horizons = list(set([c.split('_')[-1][1:] for c in df_metrics_all.columns]))
for horizon in horizons:
    print('\n----------------')
    print(f"Horizon: {horizon}")
    df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c.endswith(f"_h{horizon}")]].dropna()

    df_horizon = update_df_metrics(df_horizon,exp_i)
    df_horizon['legend_group'] = df_horizon.apply(change_legend,axis = 1)

    if configs_to_keep is not None: 
        df_horizon = df_horizon[df_horizon['id'].isin(configs_to_keep)]
        df_horizon['id'] = df_horizon['id'].apply(reduce_id)

    df_horizon['id'] = df_horizon['id'].apply(lambda x: x.replace('_In24_adp16',''))

    # plot_boxplot_on_metric(df_horizon, metric_i='mae', xaxis_label="Config", legend_group='legend_group', width=800, height=600, 
    #                             save_path=None,palette =palette ,legend_groups =legend_groups )
    title = f'RMSE Distribution per Config and Fusion Strategy of Subway-In Prediction at Horizon [{15*(int(horizon)-1)} - {15*(int(horizon))}]min'
    plot_boxplot_on_metric(df_horizon, metric_i='rmse', xaxis_label="Config", legend_group='legend_group', width=800, height=600, 
                                save_path=None,palette =palette ,legend_groups =legend_groups,title = title)
    # plot_boxplot_on_metric(df_horizon, metric_i='mase', xaxis_label="Config", legend_group='legend_group', width=1200, height=400, 
    #                             save_path=None)
    


------------------ Exp1_subway_out ---

----------------
Horizon: 1


Loading BokehJS ...


----------------
Horizon: 4


Loading BokehJS ...

In [9]:
# --- Captions Dictionary ---
captions = {
    "Exp1": r"Prediction of Subway-Out with STAEformer using contextual data 'Subway-In' according to different Integration Strategies. Metrics are averaged over 5 runs.",
    "Exp1_subway_in": r"Prediction of Subway-In with STAEformer using contextual data 'Subway-Out' according to different Integration Strategies. Metrics are averaged over 5 runs.",
    'Exp1_subway_out': r"Prediction of Subway-Out with STAEformer using contextual data 'Subway-In' according to different Integration Strategies. Metrics are averaged over 5 runs.",
}

# --- Generate and Print LaTeX Tables ---

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +15min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_in_h1, captions["Exp1_subway_in"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_out'))
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +60min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_in_h4, captions["Exp1_subway_in"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_out'))
print(latex_table_1)

print("\n" + "="*80 + "\n")



print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +15min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_out_h1, captions["Exp1_subway_out"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_in'))
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +60min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_out_h4, captions["Exp1_subway_out"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_in'))
print(latex_table_1)

print("\n" + "="*80 + "\n")


print("\n------------------------------------------------\n%%% LaTeX code for Experiment ? Horizon +?min Table %%%\n")
print("%%% LaTeX code for Experiment 1 Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_h4, captions["Exp1"], "exp1_subway_context", parse_index_exp1_2)
print(latex_table_1)

print("\n" + "="*80 + "\n")


------------------------------------------------
%%% LaTeX code for Experiment 1 Horizon +15min Table %%%



NameError: name 'df_metrics_Exp1_subway_in_h1' is not defined

In [22]:

import os 
import sys 

current_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_path, '..','..','..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.accuracy_comparison import load_trainer_ds_from_2_trials,get_predict_real_and_inputs,get_model_args
from examples.accuracy_comparison import get_rainy_indices,plot_analysis_comparison_2_config,get_previous_and_prediction,get_gain_from_mod1
import torch 
from pipeline.utils.metrics import evaluate_metrics

def get_dict_metrics_on_rainy_events(full_predict1,full_predict2,Y_true,X,args_init1,args_init2,ds2):
    h_idx = 1
    metric_list = ['rmse','mse','mae','mase','mape']
    previous,_,_,_ = get_previous_and_prediction(full_predict1,full_predict2,Y_true,X,h_idx)
    assert args_init1.horizon_step == args_init1.step_ahead, "Horizon step must be equal to step_ahead here"

    print("\nComparison on between models across all time-slots followed by comparison on Rainy Events Only")
    _,train_rainy_indices,_ = get_rainy_indices(args = args_init2,ds = ds2,training_mode = 'train')
    print(f"Number of rainy time-slots in the train set: {len(train_rainy_indices)}, i.e {len(train_rainy_indices)/len(ds2.tensor_limits_keeper.df_verif_train)*100:.2f} % of the train set")
    # ---- Plot Accuracy comparison on rainy moments only ----
    mask,rainy_indices,df_weather = get_rainy_indices(args = args_init2,ds = ds2,training_mode = 'test')
    print(f"Number of rainy time-slots in the test set: {len(rainy_indices)}, i.e {len(rainy_indices)/len(ds2.tensor_limits_keeper.df_verif_test)*100:.2f} % of the test set\n")

    dates = mask[mask].index
    masked_index = mask.reset_index(drop=True)
    masked_index = masked_index[masked_index].index


    # --- Get Prediction on rainy time-slots:
    rainy_predict1 = torch.index_select(full_predict1,0,torch.tensor(masked_index).long())
    rainy_predict2 = torch.index_select(full_predict2,0,torch.tensor(masked_index).long())
    rainy_Y_true = torch.index_select(Y_true,0,torch.tensor(masked_index).long())
    rainy_previous = torch.index_select(previous,0,torch.tensor(masked_index).long())

    dic_metric1 = evaluate_metrics(rainy_predict1,rainy_Y_true,metrics = metric_list, previous = rainy_previous,horizon_step = h_idx)
    dic_metric1 = dict(dic_metric1.copy())
    dic_metric2 = evaluate_metrics(rainy_predict2,rainy_Y_true,metrics = metric_list, previous = rainy_previous,horizon_step = h_idx)
    dic_metric2 = dict(dic_metric2.copy())
    return dic_metric1,dic_metric2



dic_contextual_data = {'bike_out': [[]],  #,['weather','calendar']
                    }

dic_fusion_strategie = {('bike_out',()): [''],
                        ('bike_out',('weather','calendar')):['early_fusion','late_fusion'],
                        }
             
dic_feature_extractor = {('bike_out',()): [''],
                        ('bike_out',('weather','calendar')):['repeat_t_proj','feature_extractor']
                        }


model_name = 'STAEformer'
epochs = 200
range_k = range(1,6)
training_mode = 'test'
modification = {'shuffle':False,
                'data_augmentation':False,
                'torch_compile': False,
                }
log = ''
Exp = 'Exp2'  # 'Exp2'

for horizon in [4]:
    for target in ['bike_out']:
        for contextual_data in dic_contextual_data[target]:
            reversed_metric = False
            fusion_strategies = dic_fusion_strategie[(target,tuple(contextual_data))]
            for fusion_strategie in fusion_strategies:
                feature_extractors = dic_feature_extractor[(target,tuple(contextual_data))]
                for feature_extractor in feature_extractors:
                    if not('weather' in contextual_data):
                        print('\n------------------------------------------------------\nContextual data:\n')
                        contextual_data1 = ['weather','calendar'] if target == 'bike_out' else ['subway_in','weather','calendar']
                        fusion_strategie1 = dic_fusion_strategie[(target,tuple(contextual_data1))][0]
                        feature_extractor1 = dic_feature_extractor[(target,tuple(contextual_data1))][0]

                        trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                        trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data1)}_{fusion_strategie1}_{feature_extractor1}__e{epochs}_h{horizon}_bis"
                        reversed_metric = True 
                    else:
                        trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                        trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis"

                    model_args,_,path_model_args,_ = get_model_args(target,model_name,save_folder_name = f'{Exp}/{target}_{model_name}')
                    ds1,ds2,args_init1,args_init2 = None, None, None, None
                    for k in range_k:
                        trial_id1_updated = f"{trial_id1}{k}_f5"
                        trial_id2_updated = f"{trial_id2}{k}_f5"

                        trainer1,trainer2,ds1,ds2,args_init1,args_init2 = load_trainer_ds_from_2_trials(trial_id1_updated,trial_id2_updated,modification = modification,
                                                                                                        model_args=model_args,
                                                                                                        path_model_args=path_model_args,
                                                                                                        ds1_init=ds1,ds2_init=ds2,
                                                                                                        args_init1=args_init1,args_init2=args_init2,
                                                                                                        )
                                                                                                        

                        full_predict1,full_predict2,Y_true,X = get_predict_real_and_inputs(trainer1,trainer2,ds1,ds2,training_mode=training_mode)

                        globals()[f"trainer1_bis{k}"] = trainer1
                        globals()[f"trainer2_bis{k}"] = trainer2
                        globals()[f"ds1_bis{k}"] = ds1
                        globals()[f"ds2_bis{k}"] = ds2
                        globals()[f"full_predict1_bis{k}"] = full_predict1
                        globals()[f"full_predict2_bis{k}"] = full_predict2

                        dic_metric1,dic_metric2 = get_dict_metrics_on_rainy_events(globals()[f"full_predict1_bis{k}"],globals()[f"full_predict2_bis{k}"],Y_true,X,args_init1,args_init2,ds2)

                        # Keep track on metric from model 1
                        if reversed_metric:
                            dic_metric2 = dict(dic_metric1.copy())
                        RMSE = dic_metric2['rmse_all']
                        MAE = dic_metric2['mae_all']
                        MASE = dic_metric2['mase_all']
                        MAPE = dic_metric2['mape_all']
                        if feature_extractor == []:
                            log += f"{model_name}_{target}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"
                        else:
                            log += f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"

                    print(log)

## Load Saved results: Check consistency (Gain MAE == Gain MASE)

In [6]:
from experiences.convert_df_to_latex import results_to_dataframe
from experiences.contextual_data_integration.A_homogenous.exp1_subway_out_results import results as results_subway_out

ref_id = 'subway_out_calendar'
df_metrics_all = results_to_dataframe(results_subway_out,get_id = True)
df_h1 = df_metrics_all[df_metrics_all.horizon == 1]
df_h1_agg = df_h1.groupby(['target','contextual_data','horizon','id']).mean().reset_index()

# --- Compute Gain on the Average of MASE
for metric in ['RMSE','MAE','MASE']:
    ref_metric = df_h1_agg[(df_h1_agg['id'] == ref_id )][metric].values[0]
    df_h1_agg[f'{metric}_Gain_%'] = df_h1_agg.apply(lambda row: '{:.4f}'.format(100*(row[metric]-ref_metric)/ref_metric), axis=1)
display(df_h1_agg)

,target,contextual_data,horizon,id,percentage,bis,RMSE,MAE,MASE,RMSE_Gain_%,MAE_Gain_%,MASE_Gain_%
0,subway_out,subway_out,1,subway_out_calendar,-1.0,3.0,36.8094,20.3930,0.6316,0.0000,0.0000,0.0000
1,subway_out,subway_out + subway in,1,subway_out_subway_in_calendar_early_fusion_ind...,-1.0,3.0,35.8406,19.7962,0.6160,-2.6319,-2.9265,-2.4699
2,subway_out,subway_out + subway in,1,subway_out_subway_in_calendar_early_fusion_s_p...,-1.0,3.0,35.8062,19.7596,0.6148,-2.7254,-3.1060,-2.6599
3,subway_out,subway_out + subway in,1,subway_out_subway_in_calendar_early_fusion_sha...,-1.0,3.0,36.0508,19.7352,0.6138,-2.0609,-3.2256,-2.8182
4,subway_out,subway_out + subway in,1,subway_out_subway_in_calendar_early_fusion_tra...,-1.0,3.0,35.7104,19.6422,0.6112,-2.9857,-3.6817,-3.2299
5,subway_out,subway_out + subway in,1,subway_out_subway_in_calendar_late_fusion_simp...,-1.0,3.0,35.8716,20.1032,0.6256,-2.5477,-1.4211,-0.9500
6,subway_out,subway_out + subway in,1,subway_out_subway_in_calendar_late_fusion_traf...,-1.0,3.0,35.3812,19.6766,0.6124,-3.8800,-3.5130,-3.0399


## Load saved model and get NEW results: 

In [30]:
trial_id1,trial_id1_updated,list(model_args['model'].keys())[0]
trial_id1_updated in model_args['model'].keys()
trial_id2

'STAEformer_subway_out_calendar____e80_h1_bis'

In [33]:
import os 
import sys 

current_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_path, '..','..','..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.accuracy_comparison import load_trainer_ds_from_2_trials,get_predict_real_and_inputs,get_model_args
from examples.accuracy_comparison import get_rainy_indices,plot_analysis_comparison_2_config,get_previous_and_prediction,get_gain_from_mod1
import torch 
from pipeline.utils.metrics import evaluate_metrics

def get_dict_metrics(full_predict1,full_predict2,Y_true,X,args_init1,args_init2,ds2):
    h_idx = 1
    metric_list = ['rmse','mse','mae','mase','mape']
    previous,_,_,_ = get_previous_and_prediction(full_predict1,full_predict2,Y_true,X,h_idx)
    assert args_init1.horizon_step == args_init1.step_ahead, "Horizon step must be equal to step_ahead here"

    dic_metric1 = evaluate_metrics(full_predict1,Y_true,metrics = metric_list, previous = previous,horizon_step = h_idx)
    dic_metric1 = dict(dic_metric1.copy())
    dic_metric2 = evaluate_metrics(full_predict2,Y_true,metrics = metric_list, previous = previous,horizon_step = h_idx)
    dic_metric2 = dict(dic_metric2.copy())
    return dic_metric1,dic_metric2



dic_contextual_data = {'subway_out': [['subway_in','calendar']] }

dic_fusion_strategie = {('subway_out',('calendar',)): [''],
                        ('subway_out',('subway_in','calendar')):['early_fusion'], 
                        # ,'late_fusion'
                        }
             
dic_feature_extractor = {('subway_out',('calendar',)): [''],
                        ('subway_out',('subway_in','calendar')):['s_proj_t_proj','independant_embedding','shared_embedding','traffic_model_backbone_InEmb24_adp16']
                        # , 'simple_embedding','traffic_model_backbone_InEmb24_adp16'
                        }


model_name = 'STAEformer'
epochs = 80
range_k = range(1,6)
training_mode = 'test'
modification = {'shuffle':False,
                'data_augmentation':False,
                'torch_compile': False,
                }
log = ''
Exp = 'Exp1_subway_out' 

for horizon in [1,4]:
    for target in ['subway_out']:
        for contextual_data in dic_contextual_data[target]:
            fusion_strategies = dic_fusion_strategie[(target,tuple(contextual_data))]
            for fusion_strategie in fusion_strategies:
                feature_extractors = dic_feature_extractor[(target,tuple(contextual_data))]
                for feature_extractor in feature_extractors:
                    print('\n------------------------------------------------------\nContextual data:\n')
                    trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                    trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis"

                    model_args,_,path_model_args,_ = get_model_args(save_folder_name = f'{Exp}/{target}_{model_name}')
                    ds1,ds2,args_init1,args_init2 = None, None, None, None
                    for k in range_k:
                        trial_id1_updated = f"{trial_id1}{k}_f5"
                        trial_id2_updated = f"{trial_id2}{k}_f5"

                        trainer1,trainer2,ds1,ds2,args_init1,args_init2 = load_trainer_ds_from_2_trials(trial_id1_updated,trial_id2_updated,modification = modification,
                                                                                                        model_args=model_args,
                                                                                                        path_model_args=path_model_args,
                                                                                                        ds1_init=ds1,ds2_init=ds2,
                                                                                                        args_init1=args_init1,args_init2=args_init2,
                                                                                                        )
                                                                                                        

                        full_predict1,full_predict2,Y_true,X = get_predict_real_and_inputs(trainer1,trainer2,ds1,ds2,training_mode=training_mode)

                        globals()[f"trainer1_bis{k}"] = trainer1
                        globals()[f"trainer2_bis{k}"] = trainer2
                        globals()[f"ds1_bis{k}"] = ds1
                        globals()[f"ds2_bis{k}"] = ds2
                        globals()[f"full_predict1_bis{k}"] = full_predict1
                        globals()[f"full_predict2_bis{k}"] = full_predict2

                        dic_metric1,dic_metric2 = get_dict_metrics(globals()[f"full_predict1_bis{k}"],globals()[f"full_predict2_bis{k}"],Y_true,X,args_init1,args_init2,ds2)


                        RMSE = dic_metric2['rmse_all']
                        MAE = dic_metric2['mae_all']
                        MASE = dic_metric2['mase_all']
                        MAPE = dic_metric2['mape_all']
                        if feature_extractor == []:
                            log += f"{model_name}_{target}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"
                        else:
                            log += f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"

                    print(log)


------------------------------------------------------
Contextual data:

Trial ID 1:  STAEformer_subway_out_calendar__e80_h1_bis1_f5
Trial ID 2:  STAEformer_subway_out_subway_in_calendar_early_fusion_s_proj_t_proj__e80_h1_bis1_f5
model_save_path for trial id1:  /home/rrochas/prediction-validation/save/K_fold_validation/training_wo_HP_tuning/Exp1_subway_out/subway_out_STAEformer/best_models/STAEformer_subway_out_calendar__e80_h1_bis1_f5.pkl
----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 35040 elts between 2019-01-01 00:00:00 and 2019-12-31 23:45:00
Invalid dates within this fold: 1233

>>>Tackle Target dataset: subway_out
   Load data from: //home/rrochas/../../data/rrochas/prediction_validation/subway_out/subway_out.csv
   Init Dataset: 'torch.Size([35040, 40]). 0 Nan values
   TRAIN contextual_ds: torch.Size([14967, 40, 7])
   VALID contextual_ds: torch.Size([4989, 40, 7])
   TEST contextual_ds: torch.Size([4989, 40, 7])

>